In [ ]:
from datasets import load_dataset
import torch

In [ ]:
print("++++++Reading the Dataset++++++++++")
dataset = load_dataset("Anonymous/Final_idiom_all",split='train') ######### please change the dataset in to the given csv file

++++++Reading the Dataset++++++++++


In [ ]:
from huggingface_hub import login

login(token='Anonymous_uihhuiguu9')

In [ ]:
system_message='''You are an polyglot, who are having exceptional linguistic and cultural domain knowledge. Also, you are an native speaker of hindi, bengali and thai.'''

In [ ]:
def format_data(sample):
    return [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_message
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": sample["image"],
                },
                {
                    "type": "text",
                    "text": sample['Actual idiom'],
                }
            ],
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": sample["Descriptive Meaning(Human Annotation)"]
                }
            ],
        },
    ]

In [ ]:
print(dataset)

Dataset({
    features: ['image', 'Actual idiom', 'Literal Translation', 'English Pronounciation', 'Idiom meaning in English', 'Descriptive Meaning(Human Annotation)'],
    num_rows: 3533
})


In [ ]:
# print(dataset[3132])
# print(len(dataset))

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x7EFF2912CAC0>, 'Actual idiom': 'นอนเป็นท้าวกุมภัณฑ์', 'Literal Translation': 'Sleep as a lord of the garuda', 'English Pronounciation': 'Non pen thao kumphan', 'Idiom meaning in English': "Keep sleeping and don't want to wake up.", 'Descriptive Meaning(Human Annotation)': 'The Thai idiom "นอนเป็นท้าวกุมภัณฑ์" translates to "sleeping like a giant" in English. It describes someone who sleeps very deeply and soundly, often implying that they are difficult to wake up. The imagery evokes a sense of a large, powerful being in a state of complete rest, undisturbed by the world around them.'}
3533


In [ ]:
print("++++++Seperating the Dataset on Lingual Basis++++++++++")
dataset_hindi = dataset.select(range(0,1277))
dataset_thai = dataset.select(range(1382,3133))
bengali_indicies = list(range(1277,1382))+list(range(3133,3533))
dataset_bengali= dataset.select(bengali_indicies)

++++++Seperating the Dataset on Lingual Basis++++++++++


In [ ]:
from datasets import Dataset,concatenate_datasets

In [ ]:
def split_dataset(dataset1):
    train_testvalid = dataset1.train_test_split(test_size=0.3, seed=42)
    train_dataset = train_testvalid['train']
    temp_dataset = train_testvalid['test']

    # Step 2: Split the remaining 30% into 2/3 (validation) and 1/3 (test)
    # 2/3 of 30% = 20%, 1/3 of 30% = 10%
    val_test = temp_dataset.train_test_split(test_size=2/3, seed=42)
    val_dataset = val_test['train']    # 20%
    test_dataset = val_test['test']    # 10%

    return train_dataset, val_dataset, test_datasets

In [ ]:
print("++++++Splitting the Dataset and Merging++++++++++")
train_dataset_hindi, val_dataset_hindi, test_dataset_hindi = split_dataset(dataset_hindi)
train_dataset_thai, val_dataset_thai, test_dataset_thai = split_dataset(dataset_thai)
train_dataset_bengali, val_dataset_bengali, test_dataset_bengali = split_dataset(dataset_bengali)


# Merging the dataset
train_dataset_final = concatenate_datasets([train_dataset_hindi,train_dataset_thai,train_dataset_bengali])
val_dataset_final = concatenate_datasets([val_dataset_hindi,val_dataset_thai,val_dataset_bengali])
test_dataset_final = concatenate_datasets([test_dataset_hindi,test_dataset_thai,test_dataset_bengali])

print(len(train_dataset_final),len(val_dataset_final),len(test_dataset_final))


dataset.save_to_disk("test_dataset")

2471 353 709


In [ ]:
print("++++++Converting the Dataset to JSON format++++++++++")
train_dataset = [format_data(sample) for sample in train_dataset]
eval_dataset = [format_data(sample) for sample in val_dataset]
test_dataset = [format_data(sample) for sample in test_dataset]

In [ ]:
print(train_dataset[2000])

[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are an polyglot, who are having exceptional linguistic and cultural domain knowledge. Also, you are an native speaker of hindi, bengali and thai.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x7EFF2837F7F0>}, {'type': 'text', 'text': 'ย้ายสำมะโนครัวไปอยู่ในคุก'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': "The Thai idiom 'ย้ายสำมะโนครัวไปอยู่ในคุก' translates to 'Moving the household registry to prison' in English. This idiom is used to describe a situation where someone is constantly in trouble or frequently ends up in jail, to the point where it seems like their permanent residence is in prison. It conveys the idea of habitual wrongdoing or a continuous cycle of criminal behavior. \n\nFor an image based on this idiom, you might depict a family or household items being moved into a prison cell, symbolizing the notion of makin

In [ ]:
# import torch
# from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor
print("++++++Importing Models++++++++++")
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor, Qwen2_5_VLProcessor
from qwen_vl_utils import process_vision_info
import torch

In [ ]:
print("++++++setting up BitsAndBytesConfig++++++++++")
from transformers import BitsAndBytesConfig

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [ ]:
print("++++++Configuring LoRA and peft++++++++++")
from peft import LoraConfig, get_peft_model

# Configure LoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

# Apply PEFT model adaptation
peft_model = get_peft_model(model, peft_config)

# Print trainable parameters
peft_model.print_trainable_parameters()

trainable params: 2,523,136 || all params: 8,294,689,792 || trainable%: 0.0304


In [ ]:
from trl import SFTConfig

# Configure training arguments
training_args = SFTConfig(
    output_dir="Qwen_2_5_Idiom_VL_Hindi",  # Directory to save the model
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    per_device_eval_batch_size=4,  # Batch size for evaluation
    gradient_accumulation_steps=8,  # Steps to accumulate gradients
    gradient_checkpointing=True,  # Enable gradient checkpointing for memory efficiency
    # Optimizer and scheduler settings
    optim="adamw_torch_fused",  # Optimizer type
    learning_rate=2e-4,  # Learning rate for training
    lr_scheduler_type="constant",  # Type of learning rate scheduler
    # Logging and evaluation
    logging_steps=10,  # Steps interval for logging
    eval_steps=10,  # Steps interval for evaluation
    eval_strategy="steps",  # Strategy for evaluation
    save_strategy="steps",  # Strategy for saving the model
    save_steps=20,  # Steps interval for saving
    metric_for_best_model="eval_loss",  # Metric to evaluate the best model
    greater_is_better=False,  # Whether higher metric values are better
    load_best_model_at_end=True,  # Load the best model after training
    # Mixed precision and gradient settings
    bf16=True,  # Use bfloat16 precision
    tf32=True,  # Use TensorFloat-32 precision
    max_grad_norm=0.3,  # Maximum norm for gradient clipping
    warmup_ratio=0.03,  # Ratio of total steps for warmup
    # Hub and reporting
    push_to_hub=True,  # Whether to push model to Hugging Face Hub
    report_to="wandb",  # Reporting tool for tracking metrics
    # Gradient checkpointing settings
    gradient_checkpointing_kwargs={"use_reentrant": False},  # Options for gradient checkpointing
    # Dataset configuration
    dataset_text_field="",  # Text field in dataset
    dataset_kwargs={"skip_prepare_dataset": True},  # Additional dataset options
    #max_seq_length=1024  # Maximum sequence length for input
)

training_args.remove_unused_columns = False  # Keep unused columns in dataset

In [ ]:
# Create a data collator to encode text and image pairs
def collate_fn(examples):
    # Get the texts and images, and apply the chat template
    texts = [processor.apply_chat_template(example, tokenize=False) for example in examples]  # Prepare texts for processing
    image_inputs = [process_vision_info(example)[0] for example in examples]  # Process the images to extract inputs

    # Tokenize the texts and process the images
    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)  # Encode texts and images into tensors

    # The labels are the input_ids, and we mask the padding tokens in the loss computation
    labels = batch["input_ids"].clone()  # Clone input IDs for labels
    labels[labels == processor.tokenizer.pad_token_id] = -100  # Mask padding tokens in labels

    # Ignore the image token index in the loss computation (model specific)
    if isinstance(processor, Qwen2_5_VLProcessor):  # Check if the processor is Qwen2VLProcessor
        image_tokens = [151652, 151653, 151655]  # Specific image token IDs for Qwen2VLProcessor
    else:
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]  # Convert image token to ID

    # Mask image token IDs in the labels
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100  # Mask image token IDs in labels

    batch["labels"] = labels  # Add labels to the batch

    return batch  # Return the prepared batch

In [1]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
    # tokenizer=processor.tokenizer,
)

ModuleNotFoundError: No module named 'trl'

In [ ]:
print("++++++Starting the training++++++++++")
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
10,27.858400,2.576223
20,17.888000,1.926543
30,13.523100,1.366189


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


In [ ]:
print("++++++Saving the Model++++++++++")
trainer.save_model(training_args.output_dir)


In [ ]:
clear_memory()
